In [1]:
from helper import Helper
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from time import time
import matplotlib.pyplot as plt

In [2]:
h = Helper()

train_it, validation_it, test_it = h.construct_data_generator_w_validation(batch_size=10)

Found 22050 images belonging to 3 classes.
Found 3150 images belonging to 3 classes.
Found 6300 images belonging to 3 classes.


In [3]:
mobilenet_v2 = MobileNetV2(include_top=False, weights=None, input_shape=(224,224,3))
mobilenet_v2.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [4]:
x=mobilenet_v2.output
x=GlobalAveragePooling2D()(x)
preds=Dense(3,activation='softmax')(x)

model = Model(inputs=mobilenet_v2.input, outputs=preds, name='cnn_topomap_classification_model_mobilenet_v2')
model.summary()

Model: "cnn_topomap_classification_model_mobilenet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________

In [5]:
opt = Adam(learning_rate=0.0001)
loss = 'categorical_crossentropy'

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [6]:
tblog_path = h.logs_directory.format(time())
tb_log = TensorBoard(log_dir = tblog_path, profile_batch=0)

ch = ModelCheckpoint('models/mobilenet/cnn_topomap_classification_model_mobilenet_v2.h5', monitor='val_accuracy', 
                     mode='max', save_best_only=True)

# es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0001, patience=5)

In [7]:
r = model.fit_generator(train_it, epochs=100, callbacks=[ch, tb_log], validation_data=validation_it)

Epoch 1/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.9277 - accuracy: 0.5527 - val_loss: 1.2057 - val_accuracy: 0.3333
Epoch 2/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.7423 - accuracy: 0.6758 - val_loss: 1.6038 - val_accuracy: 0.3333
Epoch 3/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.6422 - accuracy: 0.7265 - val_loss: 1.5735 - val_accuracy: 0.3340
Epoch 4/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.5582 - accuracy: 0.7664 - val_loss: 0.9252 - val_accuracy: 0.5616
Epoch 5/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.4974 - accuracy: 0.7946 - val_loss: 0.9439 - val_accuracy: 0.6321
Epoch 6/100
2205/2205 [==============================] - 386s 175ms/step - loss: 0.4601 - accuracy: 0.8158 - val_loss: 1.1009 - val_accuracy: 0.5578
Epoch 7/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.4140 - accuracy: 0.8337

Epoch 56/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.0669 - accuracy: 0.9756 - val_loss: 1.0168 - val_accuracy: 0.6422
Epoch 57/100
2205/2205 [==============================] - 385s 174ms/step - loss: 0.0672 - accuracy: 0.9751 - val_loss: 1.0075 - val_accuracy: 0.6990
Epoch 58/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.0680 - accuracy: 0.9755 - val_loss: 0.5668 - val_accuracy: 0.7911
Epoch 59/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.0598 - accuracy: 0.9785 - val_loss: 0.9366 - val_accuracy: 0.7238
Epoch 60/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.0608 - accuracy: 0.9766 - val_loss: 0.8679 - val_accuracy: 0.7448
Epoch 61/100
2205/2205 [==============================] - 384s 174ms/step - loss: 0.0600 - accuracy: 0.9771 - val_loss: 1.3128 - val_accuracy: 0.6295
Epoch 62/100
2205/2205 [==============================] - 385s 175ms/step - loss: 0.0573 - accuracy:

In [8]:
model_ch = h.load('models/mobilenet/cnn_topomap_classification_model_mobilenet_v2.h5')
e_ch = model_ch.evaluate_generator(test_it)
e_ch

[0.5608233786953947, 0.8474603]

In [9]:
E = model.evaluate_generator(test_it)
E

[1.466953720981329, 0.61031747]